In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier


In [102]:
train_f = pd.read_csv("../train_features.csv")
test_f = pd.read_csv("../test_features.csv")
train_l = pd.read_csv("../train_labels.csv")

In [103]:
pd.options.display.max_columns = None

In [136]:
x = pd.concat([train_f,train_l["charged_off"]], axis=1)

In [137]:
x.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,charged_off
0,43373,NaN,12000,12000,36 months,16.02%,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
1,24414,NaN,6300,6300,36 months,14.07%,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
2,46723,NaN,4500,4500,36 months,7.21%,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
3,24878,NaN,12000,12000,36 months,9.44%,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
4,39038,NaN,12000,12000,36 months,22.35%,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0


In [229]:
x["delinq_2yrs"].value_counts()

0     32059
1      4010
2       968
3       363
4       170
5        74
6        34
7        22
8        16
9         8
12        7
10        5
11        3
18        2
15        2
17        1
19        1
Name: delinq_2yrs, dtype: int64

In [139]:
x["int_rate"] = x["int_rate"].str.replace(r'%',r'').astype('float')
x["revol_util"] = x["revol_util"].str.replace(r'%',r'').astype('float')
x["term"] = x["term"].str.replace(r'months',r'').astype('float')
x["emp_length"] = x["emp_length"].str.replace(r' years',r'')
x["emp_length"] = x["emp_length"].str.replace(r' year',r'')
x["emp_length"] = x["emp_length"].str.replace(r'<',r'')
x["emp_length"] = x["emp_length"].str.replace(r'+',r'').astype('float')



In [141]:
x[["int_rate","term"]].head()

,int_rate,term
0,16.02,36.0
1,14.07,36.0
2,7.21,36.0
3,9.44,36.0
4,22.35,36.0


In [146]:
x._get_numeric_data().head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,emp_length,annual_inc,url,desc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,charged_off
0,43373,NaN,12000,12000,36.0,16.02,422.01,4.0,81000.0,NaN,NaN,12.76,0,1,48.0,NaN,7,0,4912,23.5,15,0,48.0,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,24414,NaN,6300,6300,36.0,14.07,215.54,4.0,39000.0,NaN,NaN,21.42,0,1,NaN,NaN,10,0,11876,59.4,12,0,NaN,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,46723,NaN,4500,4500,36.0,7.21,139.38,3.0,78000.0,NaN,NaN,2.17,0,2,NaN,NaN,13,0,1715,5.2,19,0,NaN,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,24878,NaN,12000,12000,36.0,9.44,384.06,10.0,62000.0,NaN,NaN,13.76,0,0,NaN,NaN,22,0,9404,16.3,33,0,NaN,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,39038,NaN,12000,12000,36.0,22.35,460.47,10.0,70000.0,NaN,NaN,14.04,4,2,13.0,NaN,18,0,17895,24.2,45,0,13.0,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [147]:
na = x.isna().sum()

In [148]:
na[na == 37745]

member_id    37745
url          37745
desc         37745
dtype: int64

In [149]:
x = x.drop(["member_id","url","desc"],axis =1)

In [150]:
x.shape

(37745, 101)

In [151]:
na = x.isna().sum() / 37745

In [152]:
x1= x.isna().sum()

In [153]:
x_no_na = x1[x1 == 0]

In [154]:
x_no_na = x_no_na.index.values

In [230]:
sorted(x_no_na)

['acc_now_delinq',
 'acc_open_past_24mths',
 'addr_state',
 'annual_inc',
 'application_type',
 'charged_off',
 'chargeoff_within_12_mths',
 'collections_12_mths_ex_med',
 'delinq_2yrs',
 'delinq_amnt',
 'disbursement_method',
 'earliest_cr_line',
 'funded_amnt',
 'grade',
 'home_ownership',
 'id',
 'initial_list_status',
 'inq_fi',
 'inq_last_12m',
 'inq_last_6mths',
 'installment',
 'int_rate',
 'loan_amnt',
 'max_bal_bc',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'open_acc',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'open_rv_12m',
 'open_rv_24m',
 'pub_rec',
 'pub_rec_bankruptcies',
 'purpose',
 'revol_bal',
 'sub_grade',
 'tax_liens',
 'term',
 'title',
 'tot_coll_amt',
 'tot_c

# to look at
dti, 

In [156]:
X = x[x_no_na].drop(columns='charged_off')

In [157]:
x["charged_off"].value_counts()/37745

0    0.852855
1    0.147145
Name: charged_off, dtype: float64

In [222]:
dti = x[ x["dti"] == np.nan]

In [228]:
x["tax_liens"].isna().sum() , x["tax_liens"].value_counts()

(0, 0    37323
 1      294
 2       81
 3       24
 4       11
 5        7
 6        3
 7        2
 Name: tax_liens, dtype: int64)

In [232]:
def mr_clean(x):
    """Cleaning data"""
    
    
    
    x["int_rate"] = x["int_rate"].str.replace(r'%',r'').astype('float')
    x["revol_util"] = x["revol_util"].str.replace(r'%',r'').astype('float')
    x["term"] = x["term"].str.replace(r'months',r'').astype('float')
    x["emp_length"] = x["emp_length"].str.replace(r' years',r'')
    x["emp_length"] = x["emp_length"].str.replace(r' year',r'')
    x["emp_length"] = x["emp_length"].str.replace(r'<',r'')
    x["emp_length"] = x["emp_length"].str.replace(r'+',r'').astype('float')
    x["emp_length"] = x["emp_length"].replace(np.nan,0)
    x["revol_util"] = x["revol_util"].replace(np.nan,0)
    
    return x

In [191]:
x["revol_util"] = x["revol_util"].replace(np.nan,0)

In [192]:
X = x[["grade","sub_grade","home_ownership","addr_state","zip_code","disbursement_method",
       "application_type", "int_rate", "revol_util", "emp_length"]]
X.isna().sum()

grade                  0
sub_grade              0
home_ownership         0
addr_state             0
zip_code               0
disbursement_method    0
application_type       0
int_rate               0
revol_util             0
emp_length             0
dtype: int64

In [209]:
xx = x["annual_inc"]
xx.isna().sum()

0

In [231]:
X = x[["grade","sub_grade","home_ownership","addr_state","zip_code","disbursement_method",
       "application_type", "int_rate", "revol_util", "emp_length", "loan_amnt","term","installment",
      "annual_inc","purpose","tax_liens", "delinq_2yrs"]]

y = x['charged_off']

ord_mapping = [{'col': 'grade', 'mapping': [("G", 0), ("F", 1), ("E", 2), ("D",3),("C",4),("B",5),("A",6)]},
              {"col" : 'sub_grade', "mapping" : [("G5", 0 ), ("G4",1), ("G3",2),("G2",3),("G1",4),
                                                ("F5", 5 ), ("F4",6), ("F3",7),("F2",8),("F1",9),
                                                ("E5", 10 ), ("E4",11), ("E3",12),("E2",13),("E1",14),
                                                ("D5", 15 ), ("D4",16), ("D3",17),("D2",18),("D1",19),
                                                ("C5", 20 ), ("C4",21), ("C3",22),("C2",23),("C1",24),
                                                ("B5", 25 ), ("B4",26), ("B3",27),("B2",28),("B1",29),
                                                ("A5", 30 ), ("A4",31), ("A3",32),("A2",33),("A1",34),
                                                ]}]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

pipe = make_pipeline(
    ce.OrdinalEncoder(mapping=ord_mapping, cols=["grade","sub_grade"]), 
    ce.OneHotEncoder(cols = ["home_ownership","addr_state","zip_code","disbursement_method","application_type","zip_code",
                             "addr_state","purpose"
                             ],
                    use_cat_names=True),
    DecisionTreeClassifier(max_depth=6)
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)

array([0.7066268 , 0.69879013, 0.69288227, 0.68961285, 0.68235984])

In [237]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['grade', 'sub_grade'], drop_invariant=False,
        handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'grade', 'mapping': [('G', 0), ('F', 1), ('E', 2), ('D', 3), ('C', 4), ('B', 5), ('A', 6)]}, {'col': 'sub_grade', 'mapping': [('G...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [234]:
x_test = mr_clean(test_f)

In [236]:
pipe

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['grade', 'sub_grade'], drop_invariant=False,
        handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'grade', 'mapping': [('G', 0), ('F', 1), ('E', 2), ('D', 3), ('C', 4), ('B', 5), ('A', 6)]}, {'col': 'sub_grade', 'mapping': [('G...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [240]:
y_test_predict = pipe.predict(x_test[["grade","sub_grade","home_ownership","addr_state","zip_code","disbursement_method",
       "application_type", "int_rate", "revol_util", "emp_length", "loan_amnt","term","installment",
      "annual_inc","purpose","tax_liens", "delinq_2yrs"]])

In [279]:
predictions = pd.concat([test_f["id"],pd.Series(y_test_predict)],axis=1)

In [280]:
predictions = predictions.rename(columns={0:"charged_off"})

In [281]:
predictions.head()

,id,charged_off
0,57146,0
1,20988,0
2,10031,0
3,33892,0
4,35374,0


In [284]:
predictions[["id","charged_off"]].to_csv("predictions.csv",index=False)

In [285]:
predictions

,id,charged_off
0,57146,0
1,20988,0
2,10031,0
3,33892,0
4,35374,0
5,54509,0
6,27804,0
7,17725,0
8,49577,0
9,55438,0
